In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE

# Parameters

In [2]:
TF2 = 2
TF3 = 3
NTest = 96
NVal = 100
T1 = 7
D = 15
NFeature = 6

# Original Dataset

In [3]:
df_original = pd.read_csv("G:/My Drive/CNTMDT/_Final/Dataset/rawTimeSeires.csv", parse_dates=['Date'], index_col=0)
df_original.head(5)

,Gold_Close,Oil_Close,Volume_Top2,Volume_Top3,Volume_Top4,Volume_Top5,X1,X2,X3,X4,X5,X6,X7,X8,X9,Y
Date,,,,,,,,,,,,,,,,
2020-03-12,1589.300049,31.500000,0.138792,0.022247,0.467458,0.033028,1.354483e+07,1.655290e+13,1.242504e+08,12.241667,1.235117,653156.0,2317.735099,349978.0,0.376633,4970.788086
2020-03-13,1515.699951,31.730000,0.130252,0.024123,0.464246,0.034736,8.634487e+06,1.655290e+13,1.028563e+08,14.216667,1.296698,517382.0,2214.384000,276798.0,0.313409,5563.707031
2020-03-14,1515.699951,31.730000,0.123606,0.022415,0.468859,0.034364,8.986521e+06,1.655290e+13,1.053248e+08,13.766667,1.274480,413935.0,2186.156250,279828.0,0.768603,5200.366211
2020-03-15,1515.699951,31.730000,0.132685,0.022928,0.458431,0.031296,8.103101e+06,1.655290e+13,9.791919e+07,12.683333,1.113645,532450.0,1875.882353,223230.0,1.172461,5392.314941
2020-03-16,1485.900024,28.700001,0.127474,0.021105,0.457094,0.032533,7.889847e+06,1.655290e+13,9.791919e+07,12.950000,1.193453,540902.0,2272.260504,270399.0,0.431327,5014.479980


Chứng minh tính seasonality: https://www.sciencedirect.com/science/article/abs/pii/S1568494621008747#:~:text=This%20study%20proposes%20a%20framework%20for%20automatic%20high-frequency,to%20align%20with%20the%20idea%20of%20reinforcement%20learning.

## Normalization

In [4]:
df = df_original.copy()

In [5]:
df_features = df.iloc[:,:df.shape[1]-1]
df_label = df.iloc[:, [df.shape[1]-1]]

In [6]:
scaler = MinMaxScaler()
scaler_single = MinMaxScaler()
scaled_data = np.concatenate([scaler.fit_transform(df_features), scaler_single.fit_transform(df_label)], axis = 1)
# scaled_data = 
# series = scaler.transform(df)
df_scaled = pd.DataFrame(data = scaled_data, columns = df.columns)
df_scaled.head(5)

,Gold_Close,Oil_Close,Volume_Top2,Volume_Top3,Volume_Top4,Volume_Top5,X1,X2,X3,X4,X5,X6,X7,X8,X9,Y
0,0.195054,0.428501,0.252108,0.035745,0.666943,0.028726,0.090175,0.096398,0.215437,0.385386,0.294139,0.478063,0.760187,0.813189,0.055378,0.000000
1,0.066875,0.429926,0.236596,0.038759,0.662361,0.030444,0.023122,0.096398,0.145379,0.473294,0.327844,0.241249,0.699492,0.549101,0.038502,0.009472
2,0.066875,0.429926,0.224523,0.036015,0.668942,0.030070,0.027929,0.096398,0.153463,0.453264,0.315684,0.060818,0.682915,0.560035,0.160002,0.003668
3,0.066875,0.429926,0.241015,0.036838,0.654064,0.026986,0.015865,0.096398,0.129212,0.405045,0.227655,0.267530,0.500701,0.355787,0.267800,0.006734
4,0.014977,0.411145,0.231549,0.033910,0.652156,0.028229,0.012953,0.096398,0.129212,0.416914,0.271336,0.282272,0.733482,0.526008,0.069977,0.000698


In [7]:
df_scaled.index = df_features.index
df_scaled.head(5)

,Gold_Close,Oil_Close,Volume_Top2,Volume_Top3,Volume_Top4,Volume_Top5,X1,X2,X3,X4,X5,X6,X7,X8,X9,Y
Date,,,,,,,,,,,,,,,,
2020-03-12,0.195054,0.428501,0.252108,0.035745,0.666943,0.028726,0.090175,0.096398,0.215437,0.385386,0.294139,0.478063,0.760187,0.813189,0.055378,0.000000
2020-03-13,0.066875,0.429926,0.236596,0.038759,0.662361,0.030444,0.023122,0.096398,0.145379,0.473294,0.327844,0.241249,0.699492,0.549101,0.038502,0.009472
2020-03-14,0.066875,0.429926,0.224523,0.036015,0.668942,0.030070,0.027929,0.096398,0.153463,0.453264,0.315684,0.060818,0.682915,0.560035,0.160002,0.003668
2020-03-15,0.066875,0.429926,0.241015,0.036838,0.654064,0.026986,0.015865,0.096398,0.129212,0.405045,0.227655,0.267530,0.500701,0.355787,0.267800,0.006734
2020-03-16,0.014977,0.411145,0.231549,0.033910,0.652156,0.028229,0.012953,0.096398,0.129212,0.416914,0.271336,0.282272,0.733482,0.526008,0.069977,0.000698


In [8]:
df_scaled.to_csv("G:/My Drive/CNTMDT/_Final/Dataset/normalization.csv")

In [108]:
import joblib
scaler_filename = "G:/My Drive/CNTMDT/_Final/Dataset/scaler.save"
joblib.dump(scaler, scaler_filename) 
scaler_single_filename = "G:/My Drive/CNTMDT/_Final/Dataset/scaler_single.save"
joblib.dump(scaler_single, scaler_single_filename) 

['G:/My Drive/CNTMDT/_Final/Dataset/scaler_single.save']

In [8]:
from statsmodels.tsa.stattools import adfuller

In [9]:
def adf_test(series, title=''):
    """
    Pass in a time series and an optional title, returns an ADF report
    """
    print(f'Augmented Dickey-Fuller Test: {title}')
    # .dropna() handles differenced data
    result = adfuller(series.dropna(), autolag='AIC')

    labels = ['ADF test statistic', 'p-value',
              'Number of lags used', 'Number of observations']
    out = pd.Series(result[0:4], index=labels)

    for key, val in result[4].items():
        out[f'critical value ({key})'] = val

    # .to_string() removes the line "dtype: float64"
    print(out.to_string())

    if result[1] <= 0.05:
        print("Strong evidence against the null hypothesis")
        print("Reject the null hypothesis")
        print("Data has no unit root and is stationary")
    else:
        print("Weak evidence against the null hypothesis")
        print("Fail to reject the null hypothesis")
        print("Data has a unit root and is non stationary")

In [10]:
adf_test(df_scaled['Y'],'BITCOIN Price')

Augmented Dickey-Fuller Test: BITCOIN Price
ADF test statistic          -1.684254
p-value                      0.439243
Number of lags used          0.000000
Number of observations    1095.000000
critical value (1%)         -3.436336
critical value (5%)         -2.864183
critical value (10%)        -2.568177
Weak evidence against the null hypothesis
Fail to reject the null hypothesis
Data has a unit root and is non stationary


In [11]:
df_scaled['DiffY'] = df_scaled['Y'].diff()


In [12]:
adf_test(df_scaled['DiffY'],'BITCOIN Price')

Augmented Dickey-Fuller Test: BITCOIN Price
ADF test statistic         -33.988249
p-value                      0.000000
Number of lags used          0.000000
Number of observations    1094.000000
critical value (1%)         -3.436342
critical value (5%)         -2.864186
critical value (10%)        -2.568179
Strong evidence against the null hypothesis
Reject the null hypothesis
Data has no unit root and is stationary


In [13]:
df_scaled.to_csv("G:/My Drive/CNTMDT/_Final/Dataset/diff.csv")

In [18]:
df_scaled['ShiftDiffY'] = df_scaled['DiffY'].shift(1)

In [19]:
df_scaled.to_csv("G:/My Drive/CNTMDT/_Final/Dataset/shift_diff.csv")